## Basic import data to test

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import tree
import numpy as np
import pickle

# import data
print('start reading')
f = open("TraData.csv")
data = np.loadtxt(f, delimiter=',')

# select data
x = data[:, 1:]  # select columns 1 through end
y = data[:, 0]   # select column 0, the result
print('done separate x and y')


start reading
done separate x and y


## Data preprocess
- first normalize data and scaling 
- then select 95 % important thing out of the data 
    - select percentile
- Use PCA to take data
- disjoint training and validation dataset

In [2]:
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.stats import pearsonr
import numpy as np

normalizer = preprocessing.Normalizer().fit(x)
X_scaled = preprocessing.RobustScaler().fit(x)
normalizer.transform(x)
X_scaled.transform(x)
print(x.shape)

#for i in range(31):
#    print(x[0][i])
X_new = SelectPercentile(percentile=95).fit_transform(x, y)
print(X_new.shape)
#for i in range(29):
#    print(X_new[0][i])

pca = PCA(n_components=29)
X_pca = pca.fit_transform(X_new)
print(X_pca.shape)
# print(pca.explained_variance_ratio_)

#train_x, valid_x,train_y,valid_y = train_test_split(X_pca,y,test_size=0.1)
#train_x_2, valid_x_2,train_y_2,valid_y_2 = train_test_split(x,y,test_size=0.1)
print('start2')


(500000, 31)
(500000, 29)
(500000, 29)
start2


## Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier

print('start random forest')
# random forest
#clf2 = RandomForestClassifier(criterion='entropy',
#                                  n_estimators=50)
clf2 = RandomForestClassifier(criterion='entropy',max_depth = 20,
                                  n_estimators=200)
#clf2 = clf2.fit(train_x_2,train_y_2)

#print('Training Acc:\n',clf2.score(train_x_2,train_y_2))
#print('Training data predict:\n',clf2.predict(train_x_2))
#print('Validate Accuracy:\n',clf2.score(valid_x_2,valid_y_2))
#print('Validate data predict:\n',clf2.predict(valid_x_2))


clf2 = clf2.fit(X_pca, y)
cv_results = cross_validate(clf2, X_pca, y, cv=3)
sorted(cv_results.keys())
# ['fit_time', 'score_time', 'test_score']
cv_results['test_score']

# Save Model:
#f = open('clf2_Random_Forest_1.pickle', 'wb')
#pickle.dump(clf2, f)
#f.close()


start random forest


array([0.70903658, 0.71174258, 0.70818883])

## Neuro network

In [4]:
from sklearn.neural_network import MLPClassifier

clf4 = MLPClassifier(activation='relu',solver='sgd',max_iter=2000,learning_rate_init=0.001,
                    hidden_layer_sizes=(45,35,35,25)) 
print('start neuro network')
#clf4 = clf4.fit(train_x,train_y)

clf4 = clf4.fit(X_pca, y)
cv_results = cross_validate(clf4, X_pca, y, cv=3)
sorted(cv_results.keys())
# ['fit_time', 'score_time', 'test_score']
cv_results['test_score']

#print('Training Acc:\n',clf4.score(train_x,train_y))
#print('Training data predict:\n',clf4.predict(train_x))
#print('Validate Accuracy:\n',clf4.score(valid_x,valid_y))
#print('Validate data predict:\n',clf4.predict(valid_x))

# Save Model:
#f = open('clf4_NN_CY_2.pickle', 'wb')
#pickle.dump(clf4, f)
#f.close()


start neuro network


array([0.74294851, 0.74620051, 0.74166297])

## KNN method

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print('start K_Neighbour_Classifier')
neigh = KNeighborsClassifier(n_neighbors=3)
neigh = neigh.fit(train_x,train_y)

print('Training Acc:\n',neigh.score(train_x,train_y))
print('Training data predict:\n',neigh.predict(train_x))
print('Validate Accuracy:\n',neigh.score(valid_x,valid_y))
print('Validate data predict:\n',neigh.predict(valid_x))

## Voting classifier

In [34]:
from sklearn.ensemble import VotingClassifier

print('start voting soft')
eclf2 = VotingClassifier(
       estimators=[('NN', clf4), ('r_f', clf2),('ada_boost',clf)],
       voting='soft')
eclf2.fit(train_x,train_y)

print('Training Acc:\n',eclf2.score(train_x,train_y))
print('Training data predict:\n',eclf2.predict(train_x))
print('Validate Accuracy:\n',eclf2.score(valid_x,valid_y))
print('Validate data predict:\n',eclf2.predict(valid_x))


# Save Model:
f = open('cy_Vote_sot1.pickle', 'wb')
pickle.dump(eclf2, f)
f.close()

start voting soft
Training Acc:
 0.9753933333333333
Training data predict:
 [0. 1. 0. ... 1. 1. 1.]
Validate Accuracy:
 0.73232
Validate data predict:
 [0. 1. 0. ... 1. 1. 0.]


## ADA BOOST

In [5]:
from sklearn.ensemble import AdaBoostClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

clf4 = MLPClassifier(activation='relu',solver='sgd',max_iter=2000,learning_rate_init=0.001,
                    hidden_layer_sizes=(45,35))
clf = AdaBoostClassifier(clf4, n_estimators=20)

clf = clf.fit(X_pca, y)
cv_results = cross_validate(clf, X_pca, y, cv=3)
sorted(cv_results.keys())
# ['fit_time', 'score_time', 'test_score']
cv_results['test_score']


#clf=clf.fit(train_x,train_y)
#print('Training Acc:\n',clf.score(train_x,train_y))
#print('Training data predict:\n',clf.predict(train_x))
#print('Validate Accuracy:\n',clf.score(valid_x,valid_y))
#print('Validate data predict:\n',clf.predict(valid_x))


# Save Model:
#f = open('cy_ada_boost.pickle', 'wb')
#pickle.dump(clf, f)
#f.close()

ValueError: MLPClassifier doesn't support sample_weight.

## read and Output file

In [2]:
import pandas as pd
import pickle
import numpy as np

from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.stats import pearsonr
import numpy as np


infile = pd.read_csv('input.csv', header=None).values
infile = np.array(infile)

x = infile
normalizer = preprocessing.Normalizer().fit(x)
X_scaled = preprocessing.RobustScaler().fit(x)
normalizer.transform(x)
X_scaled.transform(x)
print(x.shape)

#X_new = np.c_[x[:,0], x[:,2:20]]
#X_new = np.c_[X_new, x[:,21:]]
#print(X_new.shape)

#X_new = np.c_[x[:,0], x[:,2:20]]
#X_new = np.c_[X_new, x[:,21:]]
#print(X_new.shape)

#pca = PCA(n_components=29)
#X_pca = pca.fit_transform(x)
#print(X_pca.shape)

f = open('vote_hard_ensemble.pickle', 'rb')
trained_cf = pickle.load(f)
f.close()
#print(trained_cf.score(valid_x,valid_y))
np.savetxt('output.csv', trained_cf.predict(x), fmt='%d')
print('output file saved!')

(250000, 31)
output file saved!


In [35]:
import pandas as pd
import pickle
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.stats import pearsonr
import numpy as np

f = open('vote_hard_ensemble.pickle', 'rb')
trained_cf = pickle.load(f)
f.close()

#X_new = np.c_[x[:,0], x[:,2:20]]
#X_new = np.c_[X_new, x[:,21:]]
#print(X_new.shape)
#pca = PCA(n_components=29)
#X_pca = pca.fit_transform(X_new)
#print(X_pca.shape)

print(trained_cf.score(X_pca,y))

#pca = PCA(n_components=29)
#X_pca = pca.fit_transform(x)
#print(X_pca.shape)

#print(trained_cf.score(X_pca,y))

(500000, 29)
(500000, 29)
0.951086
(500000, 29)
0.497502


## Final combination ！！！！

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import tree
import numpy as np
import pickle
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

# import data
print('start reading')
f = open("TraData.csv")
data = np.loadtxt(f, delimiter=',')

# select data
x = data[:, 1:]  # select columns 1 through end
y = data[:, 0]   # select column 0, the result
print('done separate x and y')

normalizer = preprocessing.Normalizer().fit(x)
X_scaled = preprocessing.RobustScaler().fit(x)
normalizer.transform(x)
X_scaled.transform(x)
print(x.shape)

print('start random forest')
clf1 = RandomForestClassifier(criterion='entropy',max_depth = 20,
                                  n_estimators=200)
print('start neuro network')
clf2 = MLPClassifier(activation='relu',solver='sgd',max_iter=2000,learning_rate_init=0.001,
                    hidden_layer_sizes=(45,35,35)) 

clf3 = MLPClassifier(activation='relu',solver='adam', #adam, 39, 39, 74.4
                     max_iter=1000,
                     hidden_layer_sizes=(39, 39, 39),#best: 50, 50?
                     random_state=None)

print('start voting soft')
fclf = VotingClassifier(
       estimators=[('RF', clf1), ('NN1', clf2),('NN2',clf3)],
       voting='hard')
fclf.fit(x,y)
print('end voting soft')

# Save Model:
f = open('vote_hard_ensemble.pickle', 'wb')
pickle.dump(fclf, f)
f.close()
print('end saving data')


start reading
done separate x and y
(500000, 31)
start random forest
start neuro network
start voting soft
